## Customized training

In [1]:
from src.models.model import *
from src.datasets.mydataset import *

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
from classy_vision.dataset import build_dataset
from classy_vision.generic.util import load_json
from classy_vision.meters import build_meters, AccuracyMeter, VideoAccuracyMeter
from classy_vision.tasks import ClassificationTask
from classy_vision.optim import build_optimizer
from classy_vision.losses import build_loss
from classy_vision.models import build_model

In [3]:
import json
config = load_json('configs/resnet50.json')

In [7]:
# Build dataset
datasets = {}
datasets["train"] = build_dataset(config["dataset"]["train"])
datasets["test"] = build_dataset(config["dataset"]["test"])

# Build model
model = build_model(config["model"])

# Build evaluation metric
meters = build_meters(config["meters"])

# Build loss, optimizer
loss = build_loss(config["loss"])
num_epochs = config["num_epochs"]
optimizer = build_optimizer(config["optimizer"])

task = (
    ClassificationTask()
    .set_loss(loss)
    .set_model(model)
    .set_optimizer(optimizer)
    .set_meters(meters)
    .set_num_epochs(num_epochs)
) 

for phase in ["train", "test"]:
    task.set_dataset(datasets[phase], phase)

In [ ]:
import time
import os

from classy_vision.trainer import LocalTrainer
from classy_vision.hooks import CheckpointHook
from classy_vision.hooks import LossLrMeterLoggingHook

hooks = [LossLrMeterLoggingHook(log_freq=4)]

checkpoint_dir = f"./output/classy_checkpoint_{time.time()}"
os.mkdir(checkpoint_dir)
hooks.append(CheckpointHook(checkpoint_dir, input_args={}))

task = task.set_hooks(hooks)

trainer = LocalTrainer()
trainer.train(task)

## Inference

In [17]:
from classy_vision.generic.util import load_checkpoint
from classy_vision.models import ClassyModel
from classy_vision import models

# Update this with your actual directory:
checkpoint_dir = f'./output/classy_checkpoint_1609164522.3747208/model_phase-178_end.torch'
checkpoint_data = load_checkpoint(checkpoint_dir)
model = build_model(config["model"])

In [27]:
# Load state dict
model.load_state_dict(checkpoint_data['classy_state_dict']["base_model"]["model"]["trunk"])

<All keys matched successfully>

In [55]:
test_set = MyDataset(img_folder =  '/home/l/liny/ruofan/detectron2-1_website/datasets/val_imgs',
                     annot_path ='/home/l/liny/ruofan/detectron2-1_website/datasets/val_coords.txt')

In [56]:
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)

In [57]:
correct = 0
total = 0
for j, data in enumerate(test_dataloader):
    pred = model(data["input"])
    pred = F.softmax(pred, dim=-1).argmax(dim=-1).item()
    if data["target"] == pred:
        correct += 1
    total += 1

In [61]:
print("Testing accuracy: {:.2f}".format(correct / total))

Testing accuracy: 0.95
